In [2]:
import torch
import torch.nn as nn
import pyro
import pyro.distributions as dist
from pyro.infer import MCMC, NUTS
from torch.autograd import grad
import numpy as np

In [3]:

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'device: {device}')

device: cuda:0


In [4]:
# Generate synthetic sensor data

N_obs = 10
t_obs = torch.linspace(0, 5, N_obs).unsqueeze(-1).to(device)
u_true = (t_obs**2) / 2 + t_obs
noise_std = 0.1
u_obs = u_true + noise_std * torch.randn_like(u_true)

In [5]:
# Collocation points 

N_f = 100
t_f = torch.linspace(0, 5, N_f).unsqueeze(-1).to(device).requires_grad_(True)


# Initial Point

t_ic = torch.tensor([[0.0]], requires_grad=True).to(device)

In [6]:

class PINN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(1, 20)
        self.layer2 = nn.Linear(20, 20)
        self.layer3 = nn.Linear(20, 1)
        self.act = torch.tanh

    def forward(self, t):
        z = self.act(self.layer1(t))
        x = self.act(self.layer2(z))
        return self.layer3(x)

pinn = PINN().to(device)

In [7]:
# Pyro probabilistic model

def model(t_obs, u_obs, t_f,t_ic):
    
    # Priors on all network parameters

    for name, param in pinn.named_parameters():
        pyro.sample(name, dist.Normal(0., 1.).expand(param.shape).to_event(param.dim()))

    # Sensor Likelihood

    u_pred_obs = pinn(t_obs)
    pyro.sample("obs", dist.Normal(u_pred_obs.squeeze(-1), noise_std).to_event(1),obs=u_obs.squeeze(-1))


    # Physics constraint

    u_pred_f = pinn(t_f)
    du_dt = grad(u_pred_f, t_f, grad_outputs=torch.ones_like(u_pred_f), create_graph=True)[0]
    residual = du_dt - t_f
    
    # The factor strength (lambda) can be tuned:
    pyro.factor("physics", -1e3 * residual.pow(2).sum())


    # Initial‐condition residual factor

    u_pred_ic = pinn(t_ic)
    residual_ic = u_pred_ic - 0.0
    # penalize squared IC error
    pyro.factor("ic", -1e3 * residual_ic.pow(2).sum())


In [8]:
# Run HMC (NUTS)

nuts_kernel = NUTS(model)
mcmc = MCMC(nuts_kernel, num_samples=200, warmup_steps=100)
mcmc.run(t_obs, u_obs, t_f,t_ic)

Warmup:   0%|          | 0/300 [00:00, ?it/s]

RuntimeError: One of the differentiated Tensors does not require grad
     Trace Shapes:        
      Param Sites:        
     Sample Sites:        
layer1.weight dist | 20  1
             value | 20  1
  layer1.bias dist | 20   
             value | 20   
layer2.weight dist | 20 20
             value | 20 20
  layer2.bias dist | 20   
             value | 20   
layer3.weight dist |  1 20
             value |  1 20
  layer3.bias dist |  1   
             value |  1   
          obs dist | 10   
             value | 10   

In [9]:
# Extract posterior samples

samples = mcmc.get_samples()

AttributeError: 'NoneType' object has no attribute 'items'

In [ ]:
# Posterior predictive on a fine grid
t_test = torch.linspace(0, 5, 50).unsqueeze(-1).to(device)
predictions = []
for i in range(50):  # take first 50 posterior samples
    # Load sampled parameters into the network
    for name, param in pinn.named_parameters():
        param.data = samples[name][i].reshape(param.shape).to(device)
    with torch.no_grad():
        predictions.append(pinn(t_test).cpu().numpy())
predictions = np.stack(predictions, axis=0)  # shape [50, 50, 1]

mean_pred = predictions.mean(axis=0).squeeze(-1)
std_pred = predictions.std(axis=0).squeeze(-1)

In [ ]:
print("Posterior mean at test points:", mean_pred)
print("Posterior std at test points:", std_pred)